<a href="https://colab.research.google.com/github/stephenfrein/fultonlearning/blob/main/TreeModels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### SET UP SPARK ENVIRONMENT ###

!apt-get update # Update apt-get repository.
!apt-get install openjdk-8-jdk-headless -qq > /dev/null # Install Java.
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz # Download Apache Spark
!tar xf spark-3.1.1-bin-hadoop3.2.tgz # Unzip the tgz file.
!pip install -q findspark # Install findspark. Adds PySpark to the System path during runtime.

# Set environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

!ls

# Initialize findspark
import findspark
findspark.init()

# Create a PySpark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

In [ ]:
!wget --no-check-certificate "https://drive.google.com/uc?export=download&id=1T5419ivX6B3GVVMf3O-SCtFn5GohkSHW" -O bankmarketing.csv

In [ ]:
df = spark.read.csv("bankmarketing.csv", header=True)
df.show(5)
df.printSchema()

In [ ]:
# Importing the libraries
import pandas as pd
import missingno as msno

pandas_df = df.toPandas()

# Visualize missing values as a matrix
msno.matrix(pandas_df)